In [10]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [11]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0
def statistics(file_path):
    d = pd.read_csv(file_path)
    print("total rows: ",d.shape[0])
    print(' ')
    print("total columns: ",d.shape[1])
    file_size = os.path.getsize(file_path)
    print("File Size is :", file_size, "bytes")
    

Overwriting testutility.py


In [2]:
%%writefile file.yaml
file_type: csv
dataset_name: enfr
file_name: en-fr
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - en
    - fr


Overwriting file.yaml


In [5]:
%%time
df1 = pd.read_csv("en-fr.csv")

CPU times: total: 1min 37s
Wall time: 1min 38s


In [7]:
%%time
df1 = dd.read_csv("en-fr.csv")

CPU times: total: 3.05 s
Wall time: 3.14 s


In [8]:
df1.head()

,en,fr
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...
1,Site map,Plan du site
2,Feedback,Rétroaction
3,Credits,Crédits
4,Français,English


In [10]:
df1.columns = [c.replace(' ','_').lower() for c in df1.columns]
df1.columns

Index(['en', 'fr'], dtype='object')

In [12]:
import testutility as util
config_data = util.read_config_file("file.yaml")
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df1 = pd.read_csv(source_file,config_data['inbound_delimiter'])
df1.head()

c:\users\stephen williams\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3251: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


,en,fr
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...
1,Site map,Plan du site
2,Feedback,Rétroaction
3,Credits,Crédits
4,Français,English


In [13]:
if util.col_header_val(df1,config_data)==0:
    print("validation failed")

else:
    print("col validation passed")


column name and column length validation passed
col validation passed


In [13]:
df1.to_csv('en-fr.gz', index=False,sep = ",",

          compression='gzip')  

KeyboardInterrupt: 

In [14]:
import testutility as util
util.statistics("en-fr.csv")

total rows:  22520376
